In [ ]:
# Run the Db2REST Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
import json
import requests
import pandas as pd

class Db2REST():
    
    def __init__(self, RESTServiceURL):
        self.headers = {"content-type": "application/json"}
        self.RESTServiceURL = RESTServiceURL
        self.version = "/v1"
        self.API_auth = self.version + "/auth"
        self.API_makerest = self.version + "/metadata/setup"
        self.API_services = self.version + "/services/"       
        self.API_version = self.version + "/version/"      
        self.API_execsql = self.API_services + "execsql"
        self.API_monitor = self.API_services + "monitor" 
        
        self.Verify = False
        
        import urllib3
        urllib3.disable_warnings()
        
    def connectDatabase(self, dbHost, dbName, dbPort, isSSLConnection, dbUsername, dbPassword, expiryTime="300m"):
        self.dbHost = dbHost
        self.dbName = dbName
        self.dbPort = dbPort
        self.isSSLConnection = isSSLConnection
        self.dbusername = dbUsername
        self.dbpassword = dbPassword      
        self.connectionBody = {
            "dbParms": {
            "dbHost": dbHost,
            "dbName": dbName,
            "dbPort": dbPort,
            "isSSLConnection": isSSLConnection,
            "username": dbUsername,
            "password": dbPassword
            },
            "expiryTime": expiryTime
        }
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_auth), verify=self.Verify, headers=self.headers, json=self.connectionBody)
            print (response)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
        if (response.status_code == 200):
            self.token = response.json()["token"]
            print("Successfully connected and retrieved access token")
        else:
            print(response)
            print(response.json())
            print(response.json()["errors"])
    
        self.headers = {
            "authorization": f"{self.token}",
            "content-type": "application/json"
        }
        
    def getConnection(self):
        return self.connectionBody
    
    def getService(self):
        return self.RESTServiceURL
    
    def getToken(self):
        return("Token: {}".format(self.token))
    
    def getVersion(self):
        try:
            print("{}{}".format(self.RESTServiceURL,self.API_version))
            response = requests.get("{}{}".format(self.RESTServiceURL,self.API_version),verify=self.Verify)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
         
        if (response.status_code == 200):
            return response.json()['version']
        else:
            print(response)
            print(response.json()['errors'][0]['more_info'])        
        
    def runStatement(self, sql, isQuery=True, sync=True, parameters={}):
        body = {
            "isQuery": isQuery,
            "sqlStatement": sql,
            "sync": sync,
            "parameters": parameters
        }
        
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_execsql), verify=self.Verify, headers=self.headers, json=body)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
         
        if (response.status_code == 200):
            return pd.DataFrame(response.json()['resultSet'])
        elif (response.status_code == 202):
            return response.json()["id"]
        else:
            print(response.json()['errors'][0]['more_info'])
            
    def getResult(self, job_id, limit=0):
        body = {"limit": limit}
        
        try:
            response = requests.get("{}{}{}".format(self.RESTServiceURL,self.API_services,job_id), verify=self.Verify, headers=self.headers, json=body)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
  
        if (response.status_code == 200):
            json = response.json()   
            if (json['jobStatus'] == 2):
                return json['jobStatusDescription']
            elif (json['jobStatus'] == 3):
                return pd.DataFrame(json['resultSet'])               
            elif (json['jobStatus'] == 4):
                return pd.DataFrame(json['resultSet'])  
            else: 
                return json
        elif (response.status_code == 404):
            print(response.json()['errors'])  
        elif  (response.status_code == 500):
            print(response.json()['errors'][0]['more_info'])            
        else:
            print(response.json())
            
    def createServiceMetadata(self, serviceSchema="Db2REST"):
        self.serviceSchema = serviceSchema
        body = {"schema": self.serviceSchema}
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_makerest), verify=self.Verify, headers=self.headers, json=body)
            if (response.status_code == 201):
                print(response.reason)
            else:
                print(response.json())
            
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            

    def listServices(self):
        try:
            response = requests.get("{}{}".format(self.RESTServiceURL,self.API_services), verify=self.Verify, headers=self.headers)
            return pd.DataFrame(response.json()['Db2Services'])
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
    def getServiceDetails(self, serviceName, version):
        try:
            response = requests.get("{}{}{}{}".format(self.RESTServiceURL,self.API_services,"/" + serviceName,"/" + version), verify=self.Verify, headers=self.headers)
            print(response.status_code)
            if (response.status_code == 200):
                description = response.json()
                print("Input parameters:")
                print(description["inputParameters"])
                print("Result format:")
                print(description["resultSetFields"])
            else:
                print(response.json())        
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))            
            
    def createService(self, schema, serviceDescription, serviceName, sql, version, parameters=False, isQuery=True):
        if (parameters==False):
            body = {"isQuery": isQuery,
                "schema": schema,
                "serviceDescription": serviceDescription,
                "serviceName": serviceName,
                "sqlStatement": sql.replace("\n",""),
                "version": version
            } 
        else: 
            body = {"isQuery": isQuery,
                "schema": schema,
                "serviceDescription": serviceDescription,
                "serviceName": serviceName,
                "sqlStatement": sql.replace("\n",""),
                "version": version,
                "parameters": parameters
            } 
        
        try:
            response = requests.post("{}{}".format(self.RESTServiceURL,self.API_services), verify=self.Verify, headers=self.headers, json=body)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
        if (response.status_code == 201):
             print("Service: " + serviceName + " Version: " + version + " created")
        else:
            print(response.json())  
            
    def deleteService(self, serviceName, version):
        try:
            response = requests.delete("{}{}{}{}".format(self.RESTServiceURL,self.API_services,"/" + serviceName,"/" + version), verify=self.Verify, headers=self.headers)
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))
            
        if (response.status_code == 204):
            print("Service: " + serviceName + " Version: " + version + " deleted")
        else:
            print(response.json())   
            
    def callService(self, serviceName, version, parameters, sync=True):
        body = {
            "parameters": parameters,
            "sync": sync
        }
        try:
            response = requests.post("{}{}{}{}".format(self.RESTServiceURL,self.API_services,"/" + serviceName,"/" + version), verify=self.Verify, headers=self.headers, json=body)
            if (response.status_code == 200):
                return pd.DataFrame(response.json()['resultSet'])
            elif (response.status_code == 202):
                return response.json()["id"]
            else:
                print(response.json()['errors'][0]['more_info'])
                
        except Exception as e:
            if (repr(e) == "KeyError('more_info',)"): 
                print("Service not found")
            else: 
                print("Unable to call RESTful service. Error={}".format(repr(e)))
                
    def monitorJobs(self):
        try:
            response = requests.get("{}{}".format(self.RESTServiceURL,self.API_monitor), verify=self.Verify, headers=self.headers)
            if (response.status_code == 200):
                return pd.DataFrame(response.json()['MonitorServices'])
            else:
                print(response.json())    
        except Exception as e:
            print("Unable to call RESTful service. Error={}".format(repr(e)))  